# 21.离线强化学习（offline reinforcement learning）
> 在 [6.时序差分](../RL_Fundamentals/6.时序差分(temporal_difference,TD).ipynb) 中学习 **Q-learning** 算法的时候，曾区分过 **在线策略（on-policy）算法** 和 **离线策略（off-policy）算法**，这里结合 **离线强化学习算法** 再进行对比：
> ![离线强化学习和在线策略算法、离线策略算法的区别](Illustrations/离线强化学习和在线策略算法、离线策略算法的区别.jpeg)
- 无论是 **在线策略（on-policy）** 算法还是 **离线策略（off-policy）** 算法，都有一个共同点：智能体在训练过程中可以 **不断和环境交互**，得到新的反馈数据；
- 但是，在现实生活中的许多场景下，让 **尚未学习好** 的智能体和环境交互可能会 **导致危险** 发生，或是 **造成巨大损失**；
- 因此，**离线强化学习** 的目标是，在智能体 **不和环境交互** 的情况下，仅从 **已经收集好的确定的数据集** 中，通过强化学习算法得到比较好的策略；
- **模仿学习** 和 **离线强化学习** 都可以作为强化学习的 **前期策略**，为智能体提供一个更稳健的起点，减少训练的时间和所需的探索。

> 关于 **离线强化学习**领域的全景视角，可参考 [Offline Reinforcement Learning: Tutorial, Review, and Perspectives on Open Problems" (Levine et al., 2020)](https://arxiv.org/abs/2005.01643)

### 外推误差（extrapolation error）
> 研究者们最开始尝试将 **离线策略** 算法直接照搬到离线的环境下，仅仅是去掉算法中和环境交互的部分。然而，这种做法完全失败了
> 研究者进行了 3 个简单的实验：
> ![离线策略算法直接蜕变为离线强化学习算法的三个实验](Illustrations/离线策略算法直接蜕变为离线强化学习算法的三个实验.png)
- **橙色** 代表 DDPG 算法，**蓝色** 则是禁止了与环境交互
- **第一个实验**，用 DDPG 算法训练了一个智能体，并将智能体与环境交互的所有数据都记录下来，再用这些数据训练离线 DDPG 智能体；
- **第二个实验**，DDPG 算法在训练中，每次从经验回放池中采样时就将相同的数据同步训练离线 DDPG 智能体，这样两个智能体甚至连训练时用到的数据顺序都完全相同；
- **第三个实验**，DDPG 算法在训练完毕后作为专家，在环境中采集大量数据，供离线 DDPG 智能体学习。

> 可见三个实验中，离线 DDPG 智能体的表现都远远差于在线 DDPG 智能体。
> 特别是在 **第三个模仿训练实验** 中，离线智能体面对非常优秀的数据样本却什么都没学到。
> 研究者指出， 离线策略算法不能直接迁移到离线环境中的原因，是因为 **外推误差（extrapolation error）**：

- ***外推误差** 是指在机器学习、强化学习和其他建模任务中，当模型试图对 **训练数据之外的区域** 进行推断或预测时，它可能会因为没有足够的样本或经验而产生较大的误差。特别是在在 **强化学习** 任务中，状态空间可能是高维的，智能体通过某种策略进行探索收集到的数据分布(序列)可能无法完全覆盖整个状态空间。此时离线训练的模型在利用自身掌握的策略时，面临着未知复杂的真实分布去做序列决策，就很易面临 **外推误差**。*
- 数据分布内（in-distribution, **ID**），数据分布外（Out-of-Distribution，**OOD**）


为了减少 **外推误差**，离线强化学习当中的智能体的 **当前策略** 需要做到 **只访问与数据集中相似的数据**。满足这一要求的策略称 **为批量限制策略（batch-constrained policy）**。其在选择动作的时候除了需要 **最大化Q值**，还有额外的 **两个目标**：
- 最小化选择的 **动作** 与数据集中数据的距离；
- 采取动作后能到达的 **状态** 与离线数据集中状态相似。

**离线强化学习** 有 **基于模型**的方法，如 **model-based offline reinforcement learning （MOReL）** 和 **model-based offline policy optimization（MOPO）**：这一类算法的思路基本是通过模型生成更多数据，同时通过衡量模型预测的不确定性来对生成的偏离数据集的数据进行惩罚。

以下介绍两个 **无模型** 的离线强化学习算法 —— **BCQ 和 CQL**：

## 21.1 批量限制 Q-learning 算法（限制策略）
1. 对于标准的 **表格（tabular）型环境**，状态和动作空间都是离散且有限的：只需要把策略$\pi$能选择的动作以及状态转移限制在数据集$\mathcal{D}$内，就能满足上述 1+2 个目标的平衡，即表格设定下的 **批量限制 Q-learning（batch-constrained Q-learning，BCQ）算法**：
$$Q(s,a)\leftarrow(1-\alpha)Q(s,a)+\alpha(r+\gamma Q(s^{\prime},\mathrm{argmax}_{a^{\prime}\mathrm{s.t.}(s^{\prime},a^{\prime})\in\mathcal{D}}Q(s^{\prime},a^{\prime})))$$

2. 对于 **连续状态和动作环境**，情况要复杂一些，由于数据分布不可能穷尽，需要对 **两个状态动作对的距离** 有更详细地定义。对此 **BCQ** 在常规的 **Q网络$Q_{\theta}$** 之外，额外还设计了两种模型：**生成模型$G_{\omega}(s)$、扰动网络$\xi_{\phi}(s,a,\Phi)$**。三个模型进行协同，最终会形成一个 **批量限制策略**：
$$\pi(s)=\operatorname{argmax}_{a_i+\xi_\phi(s,a_i,\Phi)}Q_\theta(s,a_i+\xi_\phi(s,a_i,\Phi)),\quad\{a_i\sim G_\omega(s)\}_{i=1}^n$$
- **生成模型$G_{\omega}(s)$** 能给出与$\mathcal{D}$中数据接近的一系列动作 $a_1,\ldots,a_n$ 用于 **Q网络$Q_{\theta}$** 的训练（**生成模型** 用 **变分自动编码器（variational auto-encoder, VAE）** 实现）
- **扰动网络$\xi_{\phi}(s,a,\Phi)$** 则会对以上生成的每个动作再施加绝对值最大为 $\Phi$ 的微扰扰动，引入小范围探索，使策略能够跳出原始数据的局限而 **避免严重外推误差**。**扰动模型** 直接通过确定性策略梯度算法训练，目标也是使Q函数最大化：
$$\phi\leftarrow\mathrm{argmax}_\phi\sum_{(s,a)\in\mathcal{D}}Q_\theta(s,a+\xi_\phi(s,a,\Phi))$$

> **BCQ算法** 中的“批量（Batch）” 指的就是整个离线数据集$\mathcal{D}$，即 **限制新策略只能在这个批量数据集分布的支持范围内行动**。
> **BCQ算法** 对离线强化学习的误差分析和实验很有启发性，是 **无模型离线强化学习** 中 **限制策略** 集合算法中的经典方法。

> 更详尽的内容可参阅 **BCQ** 原文：[Stabilizing Off-Policy Q-Learning via Bootstrapping Error Reduction](https://arxiv.org/abs/1906.00949)

## 21.2 保守 Q-learning 算法（限制函数$Q$）
> 实验证明，**外推误差** 主要会导致远离数据集的点$Q$函数会被 **过高估计**，甚至常常出现$Q$值向上发散的情况。
> 因此，相较直接限制策略生成点的距离（**BCQ**），如果能用某种方法将算法中偏离数据集的点上的$Q$函数保持在很低的值，或许也是一种消除 **部分外推误差影响** 的方法，这就是 **保守 Q-learning（conservative Q-learning，CQL）算法** 的基本思想。


> **Bellman 方程** 给出了 $Q$ 函数的递归关系：
$$Q^\pi(s,a)=\mathbb{E}\left[r(s,a)+\gamma\operatorname{E}_{a^{\prime}\sim\pi(\cdot|s^{\prime})}\left[Q^\pi(s^{\prime},a^{\prime})\right]\right]$$
> 在迭代中，可以用一个 **算子（operator）** 表示这种递归更新：
$$\mathcal{B}^\pi Q(s,a):=\mathbb{E}_{s^{\prime}\sim P(\cdot|s,a)}\left[r(s,a)+\gamma\mathbb{E}_{a^{\prime}\sim\pi(\cdot|s^{\prime})}\left[Q(s^{\prime},a^{\prime})\right]\right]$$

> 普通 **离线 Q-learning** 的回归目标是最小化 **当前$Q$函数 $Q(s,a)$** 与 **Bellman 目标 $\hat{B}^\pi\hat{Q}^k(s,a)$** 之间的均方误差：
$$\hat{Q}^{k+1}\leftarrow\operatorname{argmin}_QL(Q)$$
$$\hat{Q}^{k+1}\leftarrow\mathrm{~argmin}_Q\mathbb{E}_{(s,a)\sim\mathcal{D}}\left[\left(Q(s,a)-\hat{\mathcal{B}}^\pi\hat{Q}^k(s,a)\right)^2\right]$$
- $\hat{\mathcal{B}}^\pi$带“帽子”表示采样近似而非精确期望，是在 **有限数据集$\mathcal{D}$** 上的一个近似版本：
$$\hat{\mathcal{B}}^\pi\hat{Q}^k(s,a)\:=\:r(s,a)\:+\:\gamma\:\mathbb{E}_{s^{\prime}\sim P(\cdot|s,a),\:a^{\prime}\sim\pi(\cdot|s^{\prime})}\left[\hat{Q}^k(s^{\prime},a^{\prime})\right]$$
> $a^{\prime}\sim\pi(\cdot|s^{\prime})$可能生成**数据外动作**，容易 **外推偏高**.
> 于是 **CQL** 就在标准 Q-learning 的目标函数里加了一项 **保守正则项** $\beta\mathbb{E}_{s\sim\mathcal{D},a\sim\mu(a|s)}[Q(s,a)]$，让 $Q$ 函数在未见过的动作上不要虚高，以 **缓解** 离线 RL 的 **外推误差**：
$$\hat{Q}^{k+1}\leftarrow\mathrm{~argmin}_Q\left\{\beta\operatorname{E}_{s\sim\mathcal{D},a\sim\mu(a|s)}[Q(s,a)]+\frac{1}{2}\operatorname{E}_{(s,a)\sim\mathcal{D}}\left[\left(Q(s,a)-\hat{\mathcal{B}}^\pi\hat{Q}^k(s,a)\right)^2\right]\right\}$$
- 通过对 **一组候选动作** 的 $Q$ 值取期望并惩罚其大小，迫使这些动作的 $Q$ 值整体变小。这样就能把 **数据覆盖之外OOD** 的动作的 $Q$ 值“压低”，从而减少 **外推误差/过度自信**

> 更进一步优化：
- 生成数据集的 **行为策略$\hat{\pi}_b$** 所选的动作不会被额外惩罚，保持其 $Q$ 值可信：
$$\hat{Q}^{k+1}\leftarrow\mathrm{~argmin}_Q\quad\left\{\beta\cdot(\underbrace{\mathbb{E}_{a\sim\mathcal{D},a\sim\mu(a\mid s)}[Q(s,a)]}_{\text{离线数据外的动作期望}}-\underbrace{\left\mathbb{E}_{a\sim\mathcal{D},a\sim\hat{\pi}_b (a\mid s)}[Q(s,a)]\right}_{\text{数据分布内动作期值}})+\underbrace{\frac{1}{2}\mathbb{E}_{(s,a)\sim\mathcal{D}}\left[\left(Q(s,a)-\hat{\mathcal{B}}^{\pi}\hat{Q}^{k}(s,a)\right)^{2}\right]}_{\text{标准 Bellman 误差}}\right\}$$
- 为了更好地覆盖真正的 **高估动作**，选择一个 **最坏动作分布$\mu$**，使得 **OOD动作** 的 $Q$ 值平均尽可能大，以此$Q$ 网络会捕捉 **OOD动作**，同时又被迫降低这些 **OOD动作** 的 $Q$ 值；
- 还需要约束 **分布$\mu$**，防止退化成只挑单个动作（梯度很大，训练不稳定），添加一个 **正则项$\mathcal{R}(\mu)$**：
$$\hat{Q}^{k+1}\leftarrow\mathrm{~argmin}_Q\quad \underbrace{\max_{\mu}}_{\text{最坏动作分布}} \left[\beta\cdot({\mathbb{E}_{a\sim\mathcal{D},a\sim\mu(a\mid s)}[Q(s,a)]-{\mathbb{E}_{a\sim\mathcal{D},a\sim\hat{\pi}_b (a\mid s)}[Q(s,a)])+\frac{1}{2}\mathbb{E}_{(s,a)\sim\mathcal{D}}\left[\left(Q(s,a)-\hat{\mathcal{B}}^{\pi}\hat{Q}^{k}(s,a)\right)^{2}\right]+\underbrace{\mathcal{R}(\mu)}_{\text{正则项}}\right]$$

> **正则项$\mathcal{R}(\mu)$** 可使用 **熵正则化/KL正则化**，这里使用 **KL正则化**：
$$\mathcal{R}(\mu)=-D_{KL}(\mu,\mathcal{U}(a))$$
- 优化$\max_\mu\left(\text{保守项}+\mathcal{R}(\mu)\right)$时，会鼓励 $\mu$ 接近 **参考分布 $\mathcal{U}(a)$** ，以避免出现极端动作

### 进一步简化目标函数：消除$\mu$
- **参考分布 $\mathcal{U}(a)$** 使用 **均匀分布**
- 对每个状态 $s$ 单独考虑

> 移出关于$\mu$的项：
$$\max_\mu \left\{ { \mathbb{E}_{a\sim\mu}
\begin{bmatrix}
\beta Q(s,a)
\end{bmatrix}-D_{KL}(\mu\|\mathcal{U}(a))}\right\}$$
> 又
$$-\mathbb{E}_{a\sim\mu}[\log\frac{\mu(a)}{\mathcal{U}(a)}]=\mathbb{E}_{a\sim\mu}[\log\mathcal{U}(a)]-\mathbb{E}_{a\sim\mu}[\log\mu(a)]=\mathrm{const}+H(\mu)$$
> 所以第2项等价于 **最大化熵**：
$$\max_\mu \left\{ { \mathbb{E}_{a\sim\mu}
\begin{bmatrix}
\beta Q(s,a)
\end{bmatrix}+H(\mu)}\right\}$$
> 此时再加入归一化约束 $\int\mu(a)da=1$，**拉格朗日函数** 为：
$$\mathcal{L}(\mu,\lambda)=\int\mu(a)\beta Q(s,a)da-\int\mu(a)\log\mu(a)da+\lambda\left(1-\int\mu(a)da\right)$$

> 得出 $\mu$ 分布的 **解析解**：
$$\mu^*(a|s)=\frac{\exp(\beta Q(s,a))}{\int\exp(\beta Q(s,a^{\prime}))da^{\prime}}$$
- 即 **Softmax 分布**：在保证高 $Q$ 值动作概率大的同时，尽量让动作分布分散，覆盖整个动作空间

> 代回原目标：
$$\mathbb{E}_{a\sim\mu^*}[\beta Q(s,a)]+H(\mu^*)=\log\int\exp(\beta Q(s,a))da$$
- 即 **log-sum-exp（LSE）** 技巧：一种 **平滑最大值** 计算方法，最初源于统计物理和信息论，现在广泛用于深度学习、强化学习和概率统计中，用于 **稳定数值计算和梯度优化**

> 所以 **迭代方程** 化简为：
$$\hat{Q}^{k+1}=\mathrm{argmin}_Q\left\{\underbrace{\mathbb{E}_{s\sim\mathcal{D}}\left[\log\int\exp(\beta Q(s,a))da\right]-\beta\mathbb{E}_{s,a\sim\hat{\pi}_b}[Q(s,a)]}_{\text{保守正则化已消除 }\mu}+\underbrace{\frac{1}{2}\mathbb{E}_{(s,a)\sim\mathcal{D}}\left[\left(Q(s,a)-\hat{\mathcal{B}}^\pi\hat{\boldsymbol{Q}}^k(s,a)\right)^2\right]}_{\text{Ballman 误差}}\right\}$$

> 又因为实际训练中使用 **有限动作采样**，且 **Softmax 分布** 定义为
$$\pi(a|s)=\frac{\exp(Q(s,a))}{\sum_{a^\prime}\exp(Q(s,a^\prime))}$$
> 对两边取对数：
$$\log\pi(a|s)=Q(s,a)-\log\sum_{a^{\prime}}\exp(Q(s,a^{\prime}))$$
> 有
$$\log\sum_{a^{\prime}}\exp(Q(s,a^{\prime}))=Q(s,a)-\log\pi(a|s)$$
- **“这个动作的价值超过了策略对它的信任度多少”**
> 最终 **迭代方程** ：
$$\hat{Q}^{k+1}\leftarrow\operatorname{argmin}_Q\left\{\beta\cdot\mathbb{E}_{s\sim\mathcal{D}}\left[Q(s,a)-\log\pi(a|s)-\mathbb{E}_{a\sim\hat\pi_b(a|s)}[Q(s,a)]\right]+\frac{1}{2}\mathbb{E}_{(s,a)\sim\mathcal{D}}\left[\left(Q(s,a)-\hat{\mathcal{B}}^\pi\hat{Q}^k(s,a)\right)^2\right]\right\}$$
